<a href="https://colab.research.google.com/github/deepjyotisaha85/eva4/blob/master/session2/EVA4_Session_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

Define the Model in Pytorch. We extend the nn class provided by Pytorch. The functions **__init__, forward, backward** etc. needs to be defined

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -? OUtput? RF
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(256, 512, 3)
        self.conv6 = nn.Conv2d(512, 1024, 3)
        self.conv7 = nn.Conv2d(1024, 10, 3)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        # Putting a RELU at the last layer is issue
        #x = F.relu(self.conv7(x))
        x = self.conv7(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [0]:
!pip install torchsummary

Check if GPU is available and CUDA can be used. Allocate the device for the tensor. Torch.device(). To.device() copies the data to GPU/CPU based on what device is allocated.  

In [0]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
print("use_cuda: ", use_cuda)
!nvidia-smi
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

use_cuda:  True
Sat Feb  1 08:21:10 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P0    28W /  70W |   1485MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU M

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Preparting the Dataset. Image Normalization, Training & Validation Data Generation.

In [0]:
# Manual Seed
# Sets the seed for generating random numbers.It is recommended to set a large seed, 
# i.e. a number that has a good balance of 0 and 1 bits. Avoid having many 0 bits in the seed.
# manual_seed(2147483647)
# https://pytorch.org/docs/master/torch.html?highlight=manual_seed#torch.manual_seed
#
# Use DataLoader for Pytorch
# https://pytorch.org/docs/stable/data.html
# https://pytorch.org/docs/stable/_modules/torch/utils/data/dataloader.html
#
# Use Torch Vision Transforms for Image Transformations, includes image normalization:
# https://pytorch.org/docs/stable/torchvision/transforms.html
# 
# ToTensor() works for the image, whose elements are in range 0 to 255. 
#The entire array is converted to torch tensor and then divided by 255. This is how it is forces the network to be between 0 and 1.

torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}


train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


TQDM, Define the Train and Test Functions.

In [0]:
# optimizer.zero_grad() clears x.grad for every parameter x in the optimizer. 
# It’s important to call this before loss.backward(), otherwise you’ll accumulate the gradients from multiple passes.

# loss.backward() computes dloss/dx for every parameter x which has requires_grad=True. 
# These are accumulated into x.grad for every parameter x. 

# optimizer.step updates the value of x using the gradient x.grad. 

# https://discuss.pytorch.org/t/what-does-the-backward-function-do/9944/4

from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        # get data and target values
        data, target = data.to(device), target.to(device)
        # set the gradients to zero before every training
        optimizer.zero_grad()
        # get the output predications from the model
        output = model(data)
        # calculate the loss
        loss = F.nll_loss(output, target)
        loss.backward()
        # update the gradients
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

Define the Optimizer, LR, Momentum and Loss Function??
Invoke the TRAIN and TEST Functions

In [0]:
# Optimizer
# https://pytorch.org/docs/stable/optim.html

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

for epoch in range(1, 2):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
   

loss=0.31572362780570984 batch_id=468: 100%|██████████| 469/469 [00:18<00:00, 24.89it/s]



Test set: Average loss: 0.3564, Accuracy: 8821/10000 (88%)

